In [1]:
import os

In [2]:
%pwd

'e:\\Practice\\Practice end to end project\\practice1\\research'

In [5]:
os.chdir("../")

In [6]:


%pwd

'e:\\Practice\\Practice end to end project\\practice1'

In [ ]:
## This part will go for entity > config_entity.py file 


from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
## This part will go for cofig > configuration.py file in program

from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return  data_ingestion_config

In [ ]:
## This part will go for src > components > data_ingestion.py

import os 
import urllib.request as request
from src.datascience import logger
import zipfile

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    ## Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File Already Exists..")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [28]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-28 13:12:52,982: INFO: common: Yaml File: config\config.yaml loaded successfully]
[2025-02-28 13:12:52,995: INFO: common: Yaml File: params.yaml loaded successfully]
[2025-02-28 13:12:53,005: INFO: common: Yaml File: schema.yaml loaded successfully]
[2025-02-28 13:12:53,016: INFO: common: Created Directory at: artifacts]
[2025-02-28 13:12:53,020: INFO: common: Created Directory at: artifacts/data_ingestion]


URLError: <urlopen error [Errno 11001] getaddrinfo failed>